In [20]:
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler

# Path ke direktori dataset Anda
dataset_path = 'dataset/web_scraped/'
train_dir = 'dataset/training/'
test_dir = 'dataset/testing/'

# Set your input shape and the number of classes
input_shape = (224, 224, 3)  # Adjust according to your image size
num_classes = len(os.listdir(train_dir))

# Use pre-trained MobileNetV2 model without the top (fully connected) layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Build your model using the MobileNetV2 base
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])

# Data Augmentation
datagen_train = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load training data with augmentation
batch_size = 32  # Adjust as needed
train_data = datagen_train.flow_from_directory(
    train_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

# Learning rate scheduler function
def lr_schedule(epoch, lr):
    if epoch > 30:
        lr *= 0.5e-3
    elif epoch > 20:
        lr *= 1e-3
    elif epoch > 10:
        lr *= 1e-2
    return lr

# Learning rate scheduler callback
lr_scheduler = LearningRateScheduler(lr_schedule)

# Fit the model
epochs = 10  # Adjust as needed
model.fit(train_data, epochs=epochs, steps_per_epoch=len(train_data), callbacks=[lr_scheduler])

# Save the model to a file
model_save_path = 'mobilenetv2_model.h5'
model.save(model_save_path)
print(f'Model saved to {model_save_path}')


Found 34131 images belonging to 8 classes.
Epoch 1/10
1067/1067 [==============================] - 1601s 1s/step - loss: 0.0722 - accuracy: 0.9796 - lr: 0.0010
Epoch 2/10
1067/1067 [==============================] - 1207s 1s/step - loss: 0.0356 - accuracy: 0.9902 - lr: 0.0010
Epoch 3/10
1067/1067 [==============================] - 1241s 1s/step - loss: 0.0311 - accuracy: 0.9921 - lr: 0.0010
Epoch 4/10
1067/1067 [==============================] - 1224s 1s/step - loss: 0.0308 - accuracy: 0.9919 - lr: 0.0010
Epoch 5/10
1067/1067 [==============================] - 1274s 1s/step - loss: 0.0283 - accuracy: 0.9926 - lr: 0.0010
Epoch 6/10
1067/1067 [==============================] - 1143s 1s/step - loss: 0.0248 - accuracy: 0.9938 - lr: 0.0010
Epoch 7/10
1067/1067 [==============================] - 1118s 1s/step - loss: 0.0264 - accuracy: 0.9933 - lr: 0.0010
Epoch 8/10
1067/1067 [==============================] - 1138s 1s/step - loss: 0.0232 - accuracy: 0.9943 - lr: 0.0010
Epoch 9/10
1067/1067 

TypeError: ImageDataGenerator.flow_from_directory() got an unexpected keyword argument 'steps_per_epoch'